In [1]:
import pandas as pd
import vaex
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# 400 meter coordinate accuracy
# 1923 earliest year


In [3]:
# read in obis_essential
df_path = input("Enter the file path: ")

df = vaex.open(df_path)


In [ ]:
df.describe()


In [ ]:
df = df.dropna(
    column_names=[
        "decimalLatitude",
        "decimalLongitude",
        "coordinateUncertaintyInMeters",
        "geodeticDatum",
        "species",
        "date_year",
        "basisOfRecord",
    ]
)

In [ ]:
df.describe()

In [ ]:
df.geodeticDatum.value_counts()


In [ ]:
df = df[
    df.geodeticDatum.isin(
        ["EPSG:4326", "EPSG:4326 WGS84", "EPSG: 4326 WGS 84", "WGS84_EPSG:4326"]
    )
]

In [ ]:
# find unique years in df
years = df["date_year"].unique()


In [ ]:
# for some reason have to extract to new df to continue filtering
df = df.extract()


In [ ]:
df["date_year"] = df["date_year"].astype("float32")

df["coordinateUncertaintyInMeters"] = df["coordinateUncertaintyInMeters"].astype(
    "float32"
)


In [ ]:
df = df[df.date_year >= 1923]


In [ ]:
max_year = df["date_year"].max()

min_year = df["date_year"].min()

In [ ]:
df = df[df.coordinateUncertaintyInMeters <= 400]


In [ ]:
max_meter = df["coordinateUncertaintyInMeters"].max()

min_meter = df["coordinateUncertaintyInMeters"].min()


In [ ]:
basis = df["basisOfRecord"].unique()


In [ ]:
df = df[df.basisOfRecord.isin(
    ["HumanObservation", "MachineObservation", "Occurrence"])]


In [ ]:
df["basisOfRecord"].value_counts()


In [ ]:
df["flags"].value_counts()


In [ ]:
df = df[
    "kingdom",
    "phylum",
    "class",
    "order",
    "family",
    "genus",
    "species",
    "decimalLatitude",
    "decimalLongitude",
    "license",
    "rightsHolder",
]

In [ ]:
df.describe()


In [ ]:
species_unique = df.species.unique()


In [ ]:
species_values = np.array([len(word.split()) for word in species_unique])

# find the max value in species_values
max_value = np.max(species_values)


In [ ]:
# if value at index i in species_values is greater than 2, add the value at the same index in species_unique to a new list
species_2 = []
for i in range(len(species_values)):
    if species_values[i] > 2:
        species_2.append(species_unique[i])


In [ ]:
# remove values from species that are greater than 2

for i in range(len(species_2)):
    df = df[df.species != species_2[i]]


In [ ]:
df = df.extract()

In [4]:
# recheck
species_unique = df["species"].unique()

In [ ]:
species_values = np.array([len(word.split()) for word in species_unique])


max_value = np.max(species_values)

In [ ]:
# check for strange x at beginning of species names and remove if necessary

# df = df[
#     ~df.species.isin(["×Elyhordeum schaackianum", "×Elyleymus aleuticus"])
# ]

In [ ]:
df["license"].value_counts()


In [ ]:
license = df["license"].unique()

In [ ]:
license_removal = [
    "Copyright Frank Garita Alpízar. No use without permission.",
    "Copyright Ma. Eugenia Rodríguez Vázquez. No use without permission.",
    "Copyright Whales of Guerrero (contact: Katherina Audley). No use without permission.",
    "Copyright Rémi Bigonneau. No use without permission.",
    "Copyright Nico Ransome. No use without permission.",
    "Copyright Russian Cetacean Habitat Project. No use without permission.",
    "Copyright Jodi Frediani. No use without permission.",
    "Copyright Adam A. Pack. No use without permission.",
    "Copyright Tory Kallman. No use without permission.",
    "Copyright Tasli Shaw. No use without permission.",
    "Copyright Amy Kennedy. No use without permission.",
]


In [ ]:
for i in range(len(license_removal)):
    df = df[df.license != license_removal[i]]

In [ ]:
df = df.extract()

In [ ]:
df["license"].value_counts()

In [ ]:
df.describe()

In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_clean_path = input("Enter the file path: ")
df.export_parquet(df_clean_path)
